In [1]:
import random
import gymnasium as gym
from pprint import pprint
from catanatron import Color,RandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer, DevCardRandomPlayer, DoNothingRandomPlayer,CityRandomPlayer,SettlementRandomPlayer,LongestRoadRandomPlayer
from catanatron_gym.rewards import reward_function
from catanatron_gym.features import create_sample_vector, create_sample
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json

from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
from catanatron.json import GameEncoder
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

from stable_baselines3 import PPO
from catanatron_gym.features import create_sample_vector, get_feature_ordering,create_sample
from catanatron_gym.envs.catanatron_env import from_action_space
# FEATURES = get_feature_ordering
from pprint import pprint
import random
import pandas as pd
from shaprpy import explain
from catanatron_experimental.machine_learning.players.minimax import AlphaBetaPlayer
import matplotlib.pyplot as plt
# !pip install seaborn
import seaborn as sns
import torch
import torch.nn as nn
from torch import float32
import shap.shap
from torch.autograd import Variable
import os
import re
import argparse
from rpy2.robjects.vectors import StrVector, ListVector
from rpy2.rinterface import NULL, NA

In [2]:


def run_switch_agent(env_config, agent_path="./logs/Catan_Switch_Exp/exp_015/latest_model_1000000_steps.zip"):
    best_path = "../logs/Catan_Switch_Exp/exp_014/best_model.zip"
    latest_path = "../logs/Catan_Switch_Exp/exp_004/latest_model_1000000_steps.zip"
    features = None
    try:
        env = gym.make("catanatron_gym:catanatron-switch-v1", config=env_config)
        print("Made env using config")
    except:
        env = gym.make("catanatron_gym:catanatron-switch-v1")
        print('Made env without config')

    observation, info = env.reset()
    model = PPO.load(path=agent_path, env=env)

    for ep in range(10):
        episode_data = []  # Store data for current episode
        episode_info = []  # Store info for current episode

        for step in range(1000):
            action = model.predict(observation=observation, deterministic=True)
            record = create_sample(env.game, env.p0.color)
            features = sorted(record.keys())  # Get list of feature names in same order as observation
            episode_data.append([ep, step, observation.tolist(), action[0]])
            # print(observation)
            observation, reward, terminated, truncated, info = env.step(action[0])
            done = terminated or truncated
            if done:
                # env.unwrapped.game.state()
                game_json = GameEncoder().default(env.game)
                # pprint(game_json)
                # Save the episode data and info
                episode_info.append(info) 
                save_episode_data(ep, episode_data, episode_info, features,env)
                episode_info.append(info)  # Save info at the end of each episode
                observation, info = env.reset()
                break

        # Save the episode data and info
        # save_episode_data(ep, episode_data, episode_info, features,env)

    env.close()

def save_episode_data(ep, data, info, features,env):
    # Create directory for the episode
    ep_dir = f"./games/episode_{ep}"
    os.makedirs(ep_dir, exist_ok=True)

    # Convert data to DataFrame and save as CSV
    df = pd.DataFrame(data, columns=["ep","step",'obs', 'action'])
    print(df.dtypes)
    df.to_csv(f"{ep_dir}/episode_{ep}_data.csv", index=False)

    # Save info as JSON
    with open(f"{ep_dir}/episode_{ep}_info.json", 'w') as f:
        json.dump(info, f, indent=4)

    game_json = GameEncoder().default(env.game)
    # pprint(game_json)
    print(info)
    # my_json= json.loads(game_json)
    with open(f"{ep_dir}/episode_{ep}_game.json",'w') as f:
        f.write(json.dumps(game_json, cls=GameEncoder,indent=4))
        # json.dump(game_json, f, indent=4,cls=GameEncoder)
    with open(f"{ep_dir}/episode_{ep}_game.txt",'w') as f:
        f.write(str(game_json))
        # json.dump(game_json, f, indent=4,cls=GameEncoder)
    # # Optionally save feature list
    # with open(f"{ep_dir}/episode_{ep}_features.json", 'w') as f:
    #     json.dump(features, f, indent=4)


In [15]:
config={"enemies": [AlphaBetaPlayer(Color.RED,depth=2)],
        "map_type":"TOURNAMENT"}
run_switch_agent(config)

running switch env
{'enemies': [AlphaBetaPlayer:RED(depth=2,value_fn=base_fn,prunning=False)], 'map_type': 'TOURNAMENT'}
Made env using config
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 67, 'p0_winning': False}, 'p0_stats': {'p0_vp': 3, 'p0_actual_vp': 3, 'p0_has_road': False, 'p0_has_army': False, 'p0_longest_road_length': 2, 'p0_cities': 0, 'p0_settlements': 3}, 'p1_stats': {'p1_vp': 10, 'p1_actual_vp': 10, 'p1_has_road': True, 'p1_has_army': False, 'p1_longest_road_length': 5, 'p1_cities': 3, 'p1_settlements': 2}, 'valid_actions': {'valid_actions': [289]}}]


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 62, 'p0_winning': True}, 'p0_stats': {'p0_vp': 10, 'p0_actual_vp': 10, 'p0_has_road': True, 'p0_has_army': False, 'p0_longest_road_length': 5, 'p0_cities': 4, 'p0_settlements': 0}, 'p1_stats': {'p1_vp': 5, 'p1_actual_vp': 5, 'p1_has_road': False, 'p1_has_army': True, 'p1_longest_road_length': 2, 'p1_cities': 1, 'p1_settlements': 1}, 'valid_actions': {'valid_actions': [289]}}]


...

/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 69, 'p0_winning': False}, 'p0_stats': {'p0_vp': 2, 'p0_actual_vp': 2, 'p0_has_road': False, 'p0_has_army': False, 'p0_longest_road_length': 2, 'p0_cities': 0, 'p0_settlements': 2}, 'p1_stats': {'p1_vp': 10, 'p1_actual_vp': 10, 'p1_has_road': True, 'p1_has_army': False, 'p1_longest_road_length': 6, 'p1_cities': 3, 'p1_settlements': 2}, 'valid_actions': {'valid_actions': [289, 201]}}]


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


ep         int64
step       int64
obs       object
action    object
dtype: object
[{'game_stats': {'actions_count': 0, 'num_turns': 30, 'p0_winning': True}, 'p0_stats': {'p0_vp': 10, 'p0_actual_vp': 10, 'p0_has_road': True, 'p0_has_army': False, 'p0_longest_road_length': 5, 'p0_cities': 4, 'p0_settlements': 0}, 'p1_stats': {'p1_vp': 2, 'p1_actual_vp': 2, 'p1_has_road': False, 'p1_has_army': False, 'p1_longest_road_length': 2, 'p1_cities': 0, 'p1_settlements': 2}, 'valid_actions': {'valid_actions': [289]}}]


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
